# FX Data

In [6]:
# General
import pandas as pd
from datetime import datetime
import random
import time
from dateutil.relativedelta import relativedelta

from itertools import permutations

# File download
import yfinance as yf 
from tqdm.notebook import tqdm, trange

# Custom modules
from Tools import S3

# Hidden configurations
from mySecrets import config_file, AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY, AWS_BUCKET


In [3]:
# Connect to AWS S3 storage
s3 = S3()
s3.connect(AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY)

Connected.


In [4]:
# Settings

# Define date range
date_start = '2014-01-01'
date_end = datetime.today()

# List of currencies
l_ccy = ['USD', 'EUR', 'GBP', 'CHF', 'JPY', 'SEK', 'SGD']

In [7]:
# Create all combinations of ccy symbols
df_ccy = pd.DataFrame(
    list(permutations(l_ccy, 2)),
    columns = ['ccy_from', 'ccy_to']
)

# Infer symbol
df_ccy['Symbol'] = df_ccy.ccy_from + df_ccy.ccy_to + "=X"

df_ccy.head()

,ccy_from,ccy_to,Symbol
0,USD,EUR,USDEUR=X
1,USD,GBP,USDGBP=X
2,USD,CHF,USDCHF=X
3,USD,JPY,USDJPY=X
4,USD,SEK,USDSEK=X


In [8]:
symbols = ' '.join(df_ccy.Symbol)
symbols[:50]

'USDEUR=X USDGBP=X USDCHF=X USDJPY=X USDSEK=X USDSG'

In [9]:
data = yf.download(tickers = symbols, interval = "1d", start = date_start, end = date_end, group_by = 'ticker')

[*********************100%***********************]  42 of 42 completed


In [10]:
# Check which symbols failed
df = data.stack(level = 0).rename_axis(['Date', 'Symbol']).reset_index(level = 1)

# Resetindex to make date-index a column that can be stored in files
df = df.reset_index()

# Sort columns and keep relevant ones, for now
df = df[['Symbol', 'Date', 'Adj Close', 'Open', 'High', 'Low', 'Close', 'Volume']]

# Separate by symbols
# PL 10.01.2022 -> group by Symbols
grouped = df.groupby(df['Symbol'])

In [ ]:
# Upload file to s3 storage

date = datetime.now().strftime('%Y_%m_%d')
print(date)

# store as tsv
for g in grouped.groups.keys():

    temp = grouped.get_group(g)

    s3.uploadFile(
        df = temp, 
        name = g,
        dir = 'data/fx_raw/' + date + "/",
        bucket = "pomato"
    )    